In [7]:
%reload_ext autoreload

In [13]:
# Imports

%load_ext autoreload
%autoreload 2

%matplotlib inline
import numpy as np
import random
import csv
import pandas as pd

from fastai.imports import *
from fastai.structured import *
from matplotlib import pyplot
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV
from scipy.stats import trim_mean, kurtosis
from scipy.stats.mstats import mode, gmean, hmean
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.metrics import f1_score,\
    accuracy_score, confusion_matrix,\
    precision_score, recall_score,\
    roc_curve, roc_auc_score,\
    cohen_kappa_score, mean_absolute_error,\
    precision_recall_curve, auc,\
    average_precision_score

set_plot_sizes(12,14,16)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import csv

sys.path.append('../lib')
import utils
import preprocessing

In [15]:
# Load in data

PATH = "../data/"
df_raw = pd.read_csv(f'{PATH}churn.csv', low_memory=False, 
                     parse_dates=['canceldate', 'licence_registration_date', 'golive_date'])

In [16]:
print(len(df_raw[df_raw.churned == 0]), df_raw[df_raw.churned == 0].shape)
df_processed = preprocessing.prepare_data(df_raw)
print(len(df_processed), df_processed.shape)
df_filtered = df_processed[df_processed.licence_registration_Year >2017]
df_data, y_data, nas = proc_df(df_filtered, 'churned')
print(len(df_data), df_data.shape)

2780 (2780, 38)
5889 (5889, 100)
1351 (1351, 103)


//anaconda2/envs/fastai-cpu/lib/python3.6/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
//anaconda2/envs/fastai-cpu/lib/python3.6/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
//anaconda2/envs/fastai-cpu/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [17]:
joblib_file = "churn_model.pkl"

# Load from file
churn_model = joblib.load(joblib_file)

predictions = churn_model.predict(df_data)
probability = churn_model.predict_proba(df_data)

now = datetime.datetime.now()
array = []
for i in range(len(df_filtered[df_filtered.churned == 0].username)):
    if predictions[i] == 1:
        array.append([now,df_filtered[df_filtered.churned == 0].username.iloc[i],df_filtered[df_filtered.churned == 0].url.iloc[i],df_filtered[df_filtered.churned == 0].annual_revenue.iloc[i],probability[i][1],predictions[i]])
        

print(len(df_raw[df_raw.churned == 0]), len(array))

AttributeError: 'DataFrame' object has no attribute 'url'

In [12]:
count = 0

churn_concerns = []

for i in range(len(array)):
    if array[i][4] > .7:
        count += 1
        churn_concerns.append(array[i])

In [13]:
print(count)

58


In [14]:
now = datetime.datetime.now()

sorted_array = sorted(churn_concerns, key=lambda x: x[3], reverse=True)

#for i in range(len(sorted_array)):
for i in range(30):
    print(f"{sorted_array[i][0]},{sorted_array[i][1]},{sorted_array[i][2]},{sorted_array[i][3]},{sorted_array[i][4]},{sorted_array[i][5]}")
    with open('../data/predictions.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerows(sorted_array)

writeFile.close()

2019-10-10 17:39:08.984934,N052273,logitechshop.neto.com.au,3161843.85,0.732528807651913,1
2019-10-10 17:39:08.984934,N055220,www.xdw.com.au,800290.23,0.72875622301112,1
2019-10-10 17:39:08.984934,N054211,www.funeralproducts.com.au,649310.58,0.7372354676649026,1
2019-10-10 17:39:08.984934,N052113,www.willowspoolshop.com.au,485805.76,0.7181696187071306,1
2019-10-10 17:39:08.984934,N056267,wheel-house-bik.neto.com.au,393561.83,0.7336406594662853,1
2019-10-10 17:39:08.984934,N059921,www.directcarparts.com.au,330861.34,0.7125351429677069,1
2019-10-10 17:39:08.984934,N055984,www.trailersrus.com.au,295827.07,0.7315738493486557,1
2019-10-10 17:39:08.984934,N053827,www.nordicfusion.com.au,257118.25,0.7343055314818924,1
2019-10-10 17:39:08.984934,N060326,www.trainerhub.net.au,87983.24,0.716496964353777,1
2019-10-10 17:39:08.984934,N047717,www.onthethrottle.com.au,73909.26,0.7240534578540836,1
2019-10-10 17:39:08.984934,N060976,www.apca.net.au,70305.75,0.7153848431416557,1
2019-10-10 17:39:08.98